# Impedance Calibration Test Run

### Overview:
1. Network Preperation
1. Import Matched Trace Data
2. Specify Calibration Parameters
    - Link Impedance Function
    - Turn Impedance Function
    - Objective Function
        - Exact Overlap
        - Buffer Overlap (in progress)
        - Frechet Distance (in progress)
3. Run Calibration (in progress)
4. Export Results to Examine

In [ ]:
from pathlib import Path
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
import networkx as nx
from stochopy.optimize import minimize
import stochastic_optimization
from tqdm import tqdm
import similaritymeasures
import random

from shapely.ops import LineString, MultiLineString

import sys
sys.path.insert(0,str(Path.cwd().parent))
from network.src import modeling_turns
import speedfactor

In [ ]:
import json
config = json.load((Path.cwd().parent / 'config.json').open('rb'))
calibration_fp = Path(config['project_directory']) / 'Calibration'
cycleatl_fp = Path(config['project_directory']) / 'CycleAtlanta'
matching_fp = Path(config['project_directory']) / 'Map_Matching'
network_fp = Path(config['project_directory']) / 'Network'
if calibration_fp.exists() == False:
    calibration_fp.mkdir()

# Network Preperation

In [ ]:
turns = pd.read_parquet(network_fp/'turns_df.parquet')
links = gpd.read_file(network_fp/'final_network.gpkg',layer='edges')
turn_G = modeling_turns.make_turn_graph(turns)

In [ ]:
geo_dict = dict(zip(links['linkid'],links['geometry']))

In [ ]:
#added a major/minor classification, everything else is just left as "road"
major_road = ['primary','secondary']
major_road = major_road + [item + '_link' for item in major_road]
minor_road = ['tertiary','unclassified','residential','service','trunk','living_street']
major_road = major_road + [item + '_link' for item in minor_road]
links.loc[links['highway'].isin(major_road),'link_type_new'] = 'major_road'
links.loc[links['highway'].isin(minor_road),'link_type_new'] = 'minor_road'
links.loc[links['link_type_new'].isna(),'link_type_new'] = links.loc[links['link_type_new'].isna(),'link_type']

In [ ]:
#links['link_type_new'].unique()
links['high_traffic_stress'] = links['link_type_new'] == 'major_road'

Format variables (in progress)

In [ ]:
#links['high_traffic_stress'] = links['highway'] == 'primary'
# df_edges['bike_facility_type'].value_counts()
# df_edges['high_traffic_stress'] = df_edges['bike_facility_type'].isna() & (df_edges['highway'].map(levels) > 4 | df_edges['speed limit'] > 30)

Format turn variables (in progress)

In [ ]:
turns['left'] = turns['turn_type'] == 'left'
turns['right'] = turns['turn_type'] == 'right'

In [ ]:
speedfactor.calculate_adjusted_speed(links,9)

# Specify Link Impedance Functions

## BicyclingPlus Demo Impedance Functions

Turn Only Impedance

In [ ]:
#TODO allow for certain impedance functions to be left out

In [ ]:
#have position of beta next to name of variable
betas_links = {} 
betas_turns = {
    0 : 'right',
    1 : 'left',
}

'''
Currently works with binary and numeric variables. Categoricals will have to be
cast into a different format for now.

Link impedance is weighted by the length of the link, turns are just the impedance associated
'''

#customize this function to change impedance formula
#TODO streamline process of trying out new impedance functions
def link_impedance_function(betas,beta_links,links):
    #prevent mutating the original links gdf
    links = links.copy()
    
    links['link_cost'] = links['adj_travel_time_min']
    
    return links

def turn_impedance_function(betas,beta_turns,turns):
    #use beta coefficient to calculate turn cost
    #base_turn_cost = 30 # from Lowry et al 2016 DOI: http://dx.doi.org/10.1016/j.tra.2016.02.003
    # turn_costs = {
    #     'left': betas[1] * base_turn_cost,
    #     'right': betas[1] * base_turn_cost,
    #     'straight': betas[1] * base_turn_cost
    # }
    #turns['turn_cost'] = turns['turn_type'].map(turn_costs)

    turns = turns.copy()

    turns['turn_cost'] = 0

    for key, item in beta_turns.items():
        turns['turn_cost'] = turns['turn_cost'] + (betas[key] * turns[item])

    turns['turn_cost'] = turns['turn_cost'].astype(float)

    return turns

Stree Impedance

In [ ]:
# #have position of beta next to name of variable
# betas_links = {} 
# betas_turns = {
#     0 : 'right',
#     1 : 'left',
# }

# '''
# Currently works with binary and numeric variables. Categoricals will have to be
# cast into a different format for now.

# Link impedance is weighted by the length of the link, turns are just the impedance associated
# '''

# #customize this function to change impedance formula
# #TODO streamline process of trying out new impedance functions
# def link_impedance_function(betas,beta_links,links):
#     #prevent mutating the original links gdf
#     links = links.copy()
    
#     links['link_cost'] = links['adj_travel_time_min']
    
#     return links

# def turn_impedance_function(betas,beta_turns,turns):
#     #use beta coefficient to calculate turn cost
#     base_turn_cost = 30 # from Lowry et al 2016 DOI: http://dx.doi.org/10.1016/j.tra.2016.02.003
#     # turn_costs = {
#     #     'left': betas[1] * base_turn_cost,
#     #     'right': betas[1] * base_turn_cost,
#     #     'straight': betas[1] * base_turn_cost
#     # }
#     #turns['turn_cost'] = turns['turn_type'].map(turn_costs)

#     turns = turns.copy()

#     turns['turn_cost'] = 0

#     for key, item in beta_turns.items():
#         turns['turn_cost'] = turns['turn_cost'] + (betas[key] * turns[item])

#     turns['turn_cost'] = turns['turn_cost'].astype(float)

#     return turns

## Thesis Impedance Functions

In [ ]:
# #have position of beta next to name of variable
# betas_links = {
#     0 : 'ascent_grade',
#     1 : 'high_traffic_stress',
# }

# betas_turns = {
#     2 : 'signalized',
#     3 : 'unsignalized',
#     4 : 'left',
# }

# '''
# Currently works with binary and numeric variables. Categoricals will have to be
# cast into a different format for now.

# Link impedance is weighted by the length of the link, turns are just the impedance associated
# '''

# #customize this function to change impedance formula
# #TODO streamline process of trying out new impedance functions
# def link_impedance_function(betas,beta_links,links):
#     #prevent mutating the original links gdf
#     links = links.copy()
    
#     links['attr_multiplier'] = 0

#     for key, item in beta_links.items():
#         links['attr_multiplier'] = links['attr_multiplier'] + (betas[key] * links[item])

#     links['link_cost'] = links['length_ft'] * (1 + links['attr_multiplier'])
    
#     return links

# def turn_impedance_function(betas,beta_turns,turns):
#     #use beta coefficient to calculate turn cost
#     base_turn_cost = 30 # from Lowry et al 2016 DOI: http://dx.doi.org/10.1016/j.tra.2016.02.003
#     # turn_costs = {
#     #     'left': betas[1] * base_turn_cost,
#     #     'right': betas[1] * base_turn_cost,
#     #     'straight': betas[1] * base_turn_cost
#     # }
#     #turns['turn_cost'] = turns['turn_type'].map(turn_costs)

#     turns = turns.copy()

#     turns['turn_cost'] = 0

#     for key, item in beta_turns.items():
#         turns['turn_cost'] = turns['turn_cost'] + (betas[key] * turns[item])

#     turns['turn_cost'] = turns['turn_cost'].astype(float)

#     return turns

# Import Training Set

In [ ]:
with (calibration_fp/'test_set.pkl').open('rb') as fh:
    test_set = pickle.load(fh)
with (calibration_fp/'train_set.pkl').open('rb') as fh:
    train_set = pickle.load(fh)

In [ ]:
import stochastic_optimization
from importlib import reload
reload(stochastic_optimization)

In [ ]:
ods = stochastic_optimization.match_results_to_ods(train_set)
test_ods = stochastic_optimization.match_results_to_ods(test_set)

# objective_function = stochastic_optimization.exact_overlap
# objective_function_args = {'length_dict':length_dict,'standardize':True}

objective_function = stochastic_optimization.buffer_overlap
objective_function_kwargs = {'geo_dict':geo_dict,'buffer_ft':50,'standardize':True}

#not really sure how to best set boundary conditions yet
num_of_coefs = len(betas_links) + len(betas_turns)
bounds = [[0, 50] for _ in range(0, num_of_coefs)]

In [ ]:
past_betas = []
past_vals = []
args = (
    past_betas,
    past_vals,
    betas_links,betas_turns,
    ods,train_set,
    link_impedance_function,
    turn_impedance_function,
    links,turns,turn_G,
    objective_function,
    objective_function_kwargs
)

In [ ]:
from importlib import reload
reload(stochastic_optimization)
reload(speedfactor)
start = time.time()
# args = (df_edges,turns,turn_G,matched_traces,False)
x = minimize(stochastic_optimization.impedance_calibration, bounds=bounds, args=args, method='pso', options={'maxiter':5})
end = time.time()
print(f'Took {(end-start)/60/60} hours')

In [ ]:
np.array(past_vals).min()

In [ ]:
past_betas[np.array(past_vals).argmin()]

In [ ]:
x

Create GIFs

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import imageio
from io import BytesIO

# Function to plot a GeoSeries and save the plot
def plot_geoseries(geoseries,other_geoseries,i,past_val):
    fig, ax = plt.subplots(figsize=(20, 20))
    #cx.add_basemap(ax)
    other_geoseries.plot(ax=ax,color='blue',weight=1)
    geoseries.plot(ax=ax,color='red')
    ax.set_title(f"Iter:{i} Overlap Function:{past_val}")
    ax.set_axis_off()
    img_bytes = BytesIO()
    plt.savefig(img_bytes, format='png', bbox_inches='tight')
    plt.close()
    return img_bytes.getvalue()

In [ ]:
# num_trips = 20

# for z in range(0,num_trips):

#     #choose a random tripid
#     tripid = random.choice(list(train_set.keys()))
#     start_node = train_set[tripid]['start_node']
#     end_node = train_set[tripid]['end_node']

#     matched_edges = train_set[tripid]['matched_edges']
#     matched_edges = np.array(matched_edges)
#     matched_line = MultiLineString([geo_dict[linkid] for linkid, reverse_link in matched_edges])
#     matched_line = gpd.GeoSeries(matched_line,crs='epsg:2240')
#     matched_line = matched_line.to_crs('epsg:4326')

#     modeled_lines = []

#     for betas in past_betas:
#         #update network with the correct impedances
#         stochastic_optimization.impedance_update(betas,betas_links,betas_turns,
#                                 link_impedance_function,
#                                 turn_impedance_function,
#                                 links,turns,turn_G)
#         #find shortest path
#         modeled_edges = stochastic_optimization.impedance_path(turns,turn_G,start_node,end_node)['edge_list']
#         modeled_line = MultiLineString([geo_dict[linkid] for linkid, reverse_link in modeled_edges])
#         modeled_line = gpd.GeoSeries(modeled_line,crs='epsg:2240')
#         modeled_line = modeled_line.to_crs('epsg:4326')
#         modeled_lines.append(modeled_line)

#     # List of GeoSeries (Replace this with your own GeoSeries list)
#     geoseries_list = modeled_lines

#     # Loop through the list of GeoSeries, plot each one, and save the plot
#     images = []
#     for i, geoseries in enumerate(geoseries_list):
#         past_val = past_vals[i]
#         image_bytes = plot_geoseries(geoseries,matched_line,i,past_val)
#         images.append(imageio.imread(BytesIO(image_bytes)))

#     # Path for saving the GIF
#     gif_path = f"animations/turn_calibrate_{z}.gif"

#     # Save the images as a GIF
#     imageio.mimsave(Path.cwd()/gif_path, images, format='gif', duration=2)


#### Calculate overlap for test set
Need to use best set of betas to 

In [ ]:
#link_impedance_col = "adj_travel_time_min"

#update impedances
betas = x.x
stochastic_optimization.impedance_update(betas,betas_links,betas_turns,
                          link_impedance_function,
                          turn_impedance_function,
                          links,turns,turn_G)

#find shortest path
results_dict = {(start_node,end_node):stochastic_optimization.impedance_path(turns,turn_G,start_node,end_node) for start_node, end_node in test_ods}

#calulate objective function
val_to_minimize = objective_function(test_set,results_dict,**objective_function_kwargs)
print('Test set overlap:',np.abs(np.round(val_to_minimize,2)))

## Visualize random trip

In [ ]:
# #retrieve chosen path linkids and convert them to tuple
# chosen = [tuple(row) for row in match_results[tripid]['matched_edges'].to_numpy()]
# shortest = [tuple(row) for row in match_results[tripid]['shortest_edges'].to_numpy()]
# #retrieve modeled path linkids
# modeled_edges = results_dict[(start_node,end_node)]['edge_list']

# #get geos (non-directional)
# chosen_geo = [geo_dict[linkid[0]] for linkid in chosen]
# shortest_geo = [geo_dict[linkid[0]] for linkid in shortest]
# modeled_geo = [geo_dict[linkid[0]] for linkid in modeled_edges]

# chosen_lines = gpd.GeoSeries(chosen_geo,crs='epsg:2240')
# shortest_lines = gpd.GeoSeries(shortest_geo,crs='epsg:2240')
# modeled_lines = gpd.GeoSeries(modeled_geo,crs='epsg:2240')

# stochastic_optimization.visualize_three_no_legend(chosen_lines,shortest_lines,modeled_lines)